In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/CyberLabs_ML/multi_classification_train.csv", header=0)

x_tr=train_df.iloc[0:38400,1:21] #Train Set
y_tr=train_df.iloc[0:38400,21]

x_tst=train_df.iloc[38400:48001,1:21] #Test Set
y_tst=train_df.iloc[38400:48001,21]

x_tr1=x_tr.values
y_tr1=y_tr.values


x_tst1=x_tst.values
y_tst1=y_tst.values

print(x_tr1.shape)
print(y_tr1.shape)
print(x_cv1.shape)
print(y_cv1.shape)
print(x_tst1.shape)
print(y_tst1.shape)

(28800, 20)
(28800,)
(9600, 20)
(9600,)
(9600, 20)
(9600,)


In [ ]:
def zscore_normalize_features(X):
    """
    computes  X, zcore normalized by column

    Args:
      X (ndarray (m,n))     : input data, m examples, n features

    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma

    return (X_norm, mu, sigma)

x_tr_n, x_mu1, x_sig1 = zscore_normalize_features(x_tr1)

x_tst_n, x_mu3, x_sig3 = zscore_normalize_features(x_tst1)

In [ ]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k  # Number of neighbors to consider

    # Train the model (just store the training data)
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    # Predict the class for each instance in the test set
    def predict(self, X_test):
        predictions = []
        for x_test in X_test:
            # Compute distances to all training points
            distances = [euclidean_distance(x_test, x_train) for x_train in self.X_train]

            # Get the indices of the k-nearest neighbors
            k_nearest_indices = np.argsort(distances)[:self.k]

            # Get the labels of the k-nearest neighbors
            k_nearest_labels = self.y_train[k_nearest_indices]

            # Use majority voting to predict the class
            predicted_class = np.argmax(np.bincount(k_nearest_labels))
            predictions.append(predicted_class)

        return np.array(predictions)

In [ ]:
#Instantiate and train the KNN model
knn = KNN(k=196) # APPROXIMATE SQUARE ROOT OF 38,400
knn.fit(x_tr_n, y_tr1)



In [ ]:
y_pred = knn.predict(x_tst_n)

accuracy = np.mean(y_pred == y_tst1)
print(f"Accuracy: {accuracy * 100:.2f}%")

KeyboardInterrupt: 

In [ ]:
#Calculating F1 Score (Weighted method)
p=np.zeros(5)
r=0
count_p=np.zeros(5)
count_r=np.zeros(5)
for j in range(5):
  for i in range(y_tst1.shape[0]):
    if y_tst1[i]==j:
      count_r[j]+=1
    if y_pred[i]==j:
      count_p[j]+=1
    if y_tst1[i]==y_pred[i] and y_tst1[i]==j:
      p[j]+=1

P=np.zeros(5)
R=np.zeros(5)
F_1=np.zeros(5)

for i in range(5):
  P[i]=p[i]/count_p[i]
  R[i]=p[i]/count_r[i]
  F_1[i]=(2*P[i]*R[i])/(P[i]+R[i])
print("Class Wise Precision: ",P)
print("Class Wise Recall: ",R)
print("Class wise F1 Score: ",F_1)

f=0.
for i in range(5):
  f+=F_1[i]
f=f/5
print("The average F1 Score of model on test dataset: ",f)



In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(range(500), y_tst1[:500], color='blue', label='True labels')
plt.scatter(range(500), y_pred[:500], color='red', label='Predicted labels')
plt.legend()
plt.title("True vs Predicted labels (first 500 examples)")
plt.xlabel("Sample index")
plt.ylabel("Class label")
plt.show()